## IMPORTING the required LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import nltk # Natural Language ToolKit, for stopping, stemming, lemmatization
nltk.download('punkt') # default download
nltk.download('stopwords') # downloading the stopwords to be used in the stopping process
nltk.download('wordnet') # downloading wordnet(is like imagenet for text) text data for stemming and lemmatization purpose
from nltk.corpus import stopwords # imports the stopwords we can use
from nltk.stem.wordnet import WordNetLemmatizer # Lemmatizer
import re # Regular Expression, RegEx
from sklearn.feature_extraction.text import TfidfVectorizer

### Importing tweepy which is library for accessing the Twitter API 

In [ ]:
import tweepy
from tweepy import OAuthHandler

In [ ]:
## Accessed from Developer Twitter account
API_key = 'XBH9Lt4zTpHALV1mrC0ysY5om'
API_key_Secret = 's8MaV2KAu3Ed7SfgpU5k353p9NWpaMM3XQxgz1L53HNsotOAcq'
Access_token = '1375000738934321153-9u09DmWasVu3LvPLWuWhgoKhdN3Bs2'
Access_token_secret = 'MzMUxLRZUI6OmrHnBP1dNZisFdM96Ro3jdt2jgugYOYYg'

In [ ]:
auth = tweepy.OAuthHandler(API_key,API_key_Secret)
auth.set_access_token(Access_token,Access_token_secret)
api=tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
## Creating the dataframe 
df = pd.DataFrame(columns=["Date","User","IsVerified","Tweet","Likes","RT",'User_location'])
df1 = pd.DataFrame(columns=["Date","User","IsVerified","Tweet","Likes","RT",'User_location'])
df2 = pd.DataFrame(columns=["Date","User","IsVerified","Tweet","Likes","RT",'User_location'])

## Extracting 3000 tweets from
### 1) Amazon India
### 2) Flipkart
### 3) Snapdeal

In [ ]:
## Amazon
def get_tweets(username):
    i=0
    for tweet in tweepy.Cursor(api.search, q=username, lang="en",exclude='retweets').items(3000):
        print(i, end='\r')
        df.loc[i,"Date"] = tweet.created_at
        df.loc[i,"User"] = tweet.user.name
        df.loc[i,"IsVerified"] = tweet.user.verified
        df.loc[i,"Tweet"] = tweet.text
        df.loc[i,"Likes"] = tweet.favorite_count
        df.loc[i,"RT"] = tweet.retweet_count
        df.loc[i,"User_location"] = tweet.user.location
        df.to_csv("AmazonDataset.csv",index=False)
        #df.to_excel('{}.xlsx'.format("TweetDataset"),index=False)   ## Save as Excel
        i=i+1
        if i>=3000:
            break
        else:
            pass

get_tweets('AmazonIN')

In [ ]:
## Flipkart
def get_tweets(username):
    i=0
    for tweet in tweepy.Cursor(api.search, q=username, lang="en",exclude='retweets').items(3000):
        print(i, end='\r')
        df1.loc[i,"Date"] = tweet.created_at
        df1.loc[i,"User"] = tweet.user.name
        df1.loc[i,"IsVerified"] = tweet.user.verified
        df1.loc[i,"Tweet"] = tweet.text
        df1.loc[i,"Likes"] = tweet.favorite_count
        df1.loc[i,"RT"] = tweet.retweet_count
        df1.loc[i,"User_location"] = tweet.user.location
        df1.to_csv("FlipkartDataset.csv",index=False)
        #df.to_excel('{}.xlsx'.format("TweetDataset"),index=False)   ## Save as Excel
        i=i+1
        if i>=3000:
            break
        else:
            pass
        
get_tweets('Flipkart')

In [ ]:
## Sanpdeal
def get_tweets(username):
    i=0
    for tweet in tweepy.Cursor(api.search, q=username, lang="en",exclude='retweets').items(3000):
        print(i, end='\r')
        df2.loc[i,"Date"] = tweet.created_at
        df2.loc[i,"User"] = tweet.user.name
        df2.loc[i,"IsVerified"] = tweet.user.verified
        df2.loc[i,"Tweet"] = tweet.text
        df2.loc[i,"Likes"] = tweet.favorite_count
        df2.loc[i,"RT"] = tweet.retweet_count
        df2.loc[i,"User_location"] = tweet.user.location
        df2.to_csv("SnapdealDataset.csv",index=False)
        #df.to_excel('{}.xlsx'.format("TweetDataset"),index=False)   ## Save as Excel
        i=i+1
        if i>=3000:
            break
        else:
            pass
        
get_tweets('snapdeal')

In [ ]:
## Reading the data
df = pd.read_csv('AmazonDataset.csv')
df1 = pd.read_csv('FlipkartDataset.csv')
df2 = pd.read_csv('SnapdealDataset.csv')

In [ ]:
## Dropping the non required columns
Tweet_df = df.drop(['Date','User','IsVerified','Likes','RT','User_location'],1,inplace=True)
Tweet_df1 = df1.drop(['Date','User','IsVerified','Likes','RT','User_location'],1,inplace=True)
Tweet_df2 = df2.drop(['Date','User','IsVerified','Likes','RT','User_location'],1,inplace=True)

## Treating Punctuations and Cleaning the data.

In [ ]:
import string
result = string.punctuation
result

In [ ]:
def clean_text(Tweet):
    Tweet = Tweet.lower()
    Tweet = re.sub('\[.*?\]', '', Tweet)
    Tweet = re.sub('https?://\S+|www\.\S+', '', Tweet)
    Tweet = re.sub('<.*?>+', '', Tweet)
    Tweet = re.sub('[%s]' % re.escape(string.punctuation), '', Tweet)
    Tweet = re.sub('\n', '', Tweet)
    Tweet = re.sub('\w*\d\w*', '', Tweet)
    return Tweet

In [ ]:
Tweet_df = df['Tweet'] = df['Tweet'].apply(lambda x: clean_text(x))
Tweet_df1 = df1['Tweet'] = df1['Tweet'].apply(lambda x: clean_text(x))
Tweet_df2 = df2['Tweet'] = df2['Tweet'].apply(lambda x: clean_text(x))
df.head(5)

## Performing Positive, Negative, Neutral Tweet Analysis

In [ ]:
## Importing Textblob for sentiment analysis
from textblob import TextBlob

def analyze_sentiment(Tweet):
    analysis = TextBlob(Tweet)
    
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [ ]:
df["Sentiment"] = df["Tweet"].apply(lambda x : analyze_sentiment(x))
df1["Sentiment"] = df1["Tweet"].apply(lambda x : analyze_sentiment(x))
df2["Sentiment"] = df2["Tweet"].apply(lambda x : analyze_sentiment(x))
df.head(10)

In [ ]:
## Count for every sentiment for every tweet.

print(df["Sentiment"].value_counts())
print(df1["Sentiment"].value_counts())
print(df2["Sentiment"].value_counts())

### Creating PieChart of Sentiment analysis for all three Companies.

In [ ]:
a=len(df[df["Sentiment"]=="Positive"])
b=len(df[df["Sentiment"]=="Negative"])
c=len(df[df["Sentiment"]=="Neutral"])
d=np.array([a,b,c])
explode = (0.1, 0.0, 0.1)
plt.pie(d,shadow=True,explode=explode,labels=["Positive","Negative","Neutral"],autopct='%1.2f%%')
plt.title("Amazon India");

In [ ]:
a=len(df1[df1["Sentiment"]=="Positive"])
b=len(df1[df1["Sentiment"]=="Negative"])
c=len(df1[df1["Sentiment"]=="Neutral"])
d=np.array([a,b,c])
explode = (0.1, 0.0, 0.1)
plt.pie(d,shadow=True,explode=explode,labels=["Positive","Negative","Neutral"],autopct='%1.2f%%')
plt.title("Flipkart");

In [ ]:
a=len(df2[df2["Sentiment"]=="Positive"])
b=len(df2[df2["Sentiment"]=="Negative"])
c=len(df2[df2["Sentiment"]=="Neutral"])
d=np.array([a,b,c])
explode = (0.1, 0.0, 0.1)
plt.pie(d,shadow=True,explode=explode,labels=["Positive","Negative","Neutral"],autopct='%1.2f%%')
plt.title("Snapdeal");

## Removing stopwords

In [ ]:
stopword = nltk.corpus.stopwords.words('english')
stopword

In [ ]:
def remove_stopwords(Tweet):
    words = [w for w in Tweet if w not in stopword]
    return words

In [ ]:
df['Tweet_nonstop'] = df['Tweet'].apply(lambda x: remove_stopwords(x))
df1['Tweet_nonstop'] = df1['Tweet'].apply(lambda x: remove_stopwords(x))
df2['Tweet_nonstop'] = df2['Tweet'].apply(lambda x: remove_stopwords(x))
df1.head(10)

## Analysing the Sentiments for the products

In [ ]:
## Importing Wordcloud

from wordcloud import WordCloud, STOPWORDS

## POSITIVE REVIEWS

In [ ]:
## AMAZON IN

# POSITIVE reviews 
text_positive = " ".join(review for review in df[df["Sentiment"]=="Positive"].Tweet)
print ("There are {} words in the combination of all review.".format(len(text_positive)))
# Create stopword list:
stopwords = set(STOPWORDS)
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords,max_words=800,max_font_size=70).generate(text_positive)
plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Positive reviews of Amazon India')
plt.axis("off")
plt.show()

In [ ]:
## FLIPKART

# POSITIVE reviews 
text_positive = " ".join(review for review in df1[df1["Sentiment"]=="Positive"].Tweet)
print ("There are {} words in the combination of all review.".format(len(text_positive)))
# Create stopword list:
stopwords = set(STOPWORDS)
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords,max_words=800,max_font_size=70).generate(text_positive)
plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Positive reviews of Flipkart')
plt.axis("off")
plt.show()

In [ ]:
## SNAPDEAL

# POSITIVE reviews 
text_positive = " ".join(review for review in df2[df2["Sentiment"]=="Positive"].Tweet)
print ("There are {} words in the combination of all review.".format(len(text_positive)))
# Create stopword list:
stopwords = set(STOPWORDS)
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords,max_words=800,max_font_size=70).generate(text_positive)
plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Positive reviews of Snapdeal')
plt.axis("off")
plt.show()

## NEGATIVE REVIEWS

In [ ]:
## AMAZON IN

# Negative reviews
text_negative = " ".join(review for review in df[df["Sentiment"]=="Negative"].Tweet)
print ("There are {} words in the combination of all review.".format(len(text_negative)))
# Create stopword list:
stopwords = set(STOPWORDS)
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords,max_words=800,max_font_size=70).generate(text_negative)
plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Negative reviews - Amazon India')
plt.axis("off")
plt.show()

In [ ]:
## FLIPKART

# Negative reviews
text_negative = " ".join(review for review in df1[df1["Sentiment"]=="Negative"].Tweet)
print ("There are {} words in the combination of all review.".format(len(text_negative)))
# Create stopword list:
stopwords = set(STOPWORDS)
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords,max_words=800,max_font_size=70).generate(text_negative)
plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Negative reviews - Flipkart ')
plt.axis("off")
plt.show()

In [ ]:
## SNAPDEAL

# Negative reviews
text_negative = " ".join(review for review in df2[df2["Sentiment"]=="Negative"].Tweet)
print ("There are {} words in the combination of all review.".format(len(text_negative)))
# Create stopword list:
stopwords = set(STOPWORDS)
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords,max_words=800,max_font_size=70).generate(text_negative)
plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Negative reviews - Snapdeal')
plt.axis("off")
plt.show()

## CREATING A WORDCLOUD

### AMAZON IN

In [ ]:
# Combine all reviews 
text = " ".join(review for review in df.Tweet)
print ("There are {} words in the combination of all review.".format(len(text)))
# Create stopword list:
stopwords = set(STOPWORDS)
#stopwords.update(["drink", "now", "wine", "flavor", "flavors"])  #To add any custom StopWords
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords,max_words=3000,max_font_size=70).generate(text)
# Display the generated image:
# the matplotlib way:
plt.figure(figsize=(12,8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Reviews of Amazon India')
plt.axis("off")
plt.show()

### FLIPKART

In [ ]:
# Combine all reviews 
text = " ".join(review for review in df1.Tweet)
print ("There are {} words in the combination of all review.".format(len(text)))
# Create stopword list:
stopwords = set(STOPWORDS)
#stopwords.update(["drink", "now", "wine", "flavor", "flavors"])  #To add any custom StopWords
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords,max_words=3000,max_font_size=70).generate(text)
# Display the generated image:
# the matplotlib way:
plt.figure(figsize=(12,8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Reviews of Flipkart')
plt.axis("off")
plt.show()

### SNAPDEAL

In [ ]:
# Combine all reviews 
text = " ".join(review for review in df2.Tweet)
print ("There are {} words in the combination of all review.".format(len(text)))
# Create stopword list:
stopwords = set(STOPWORDS)
#stopwords.update(["drink", "now", "wine", "flavor", "flavors"])  #To add any custom StopWords
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords,max_words=3000,max_font_size=70).generate(text)
# Display the generated image:
# the matplotlib way:
plt.figure(figsize=(12,8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Reviews of Snapdeal')
plt.axis("off")
plt.show()

#### Final Statement: 
    1. Extracted 3000 tweets from twitter for AMAZONIN, FLIPKART, SNAPDEAL
    2. Removed stop words and Performed positive and negative tweet analysis.
    3. Created  word-cloud for all three companies.
    4. Analysed for which products what are the sentiments (by creating a wordcloud)

# THANK YOU!!!